In [321]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import json
from copy import deepcopy

In [322]:
RUN_BASELINE_ONLY = False
drop_percentages = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]
results = {}
number_of_generated_subgraphs = 100

In [323]:
for i, drop_pct in enumerate(drop_percentages):
    # The baseline does not change since there are no nodes dropped
    # so i wait for the first iteration to finish to create the first basline.log file
    # then i extract the baseline from that logfile once and let the iterations finish
    if i == 1:
        baseline_file = "experiment_3"
        baseline_success = None
        if os.path.exists(baseline_file+'.log'):
            with open(baseline_file+'.log', 'r') as f:
                content = f.read()
            baseline_match = re.search(r'Attacker can guarantee success probability of: ([0-9.]+)', content)
            if baseline_match:
                baseline_success = float(baseline_match.group(1))
            print(f"Baseline attacker success: {baseline_success}")

    print(f"\n{'='*50}")
    print(f"RUNNING EXPERIMENT WITH DROP PERCENTAGE: {drop_pct*100}%")
    print(f"{'='*50}\n")
    
    # Set variables that will be accessible in the experiment notebook
    experiment_name = f"experiment_drop_{int(drop_pct*100)}"
    # ????
    # drop_percentage_list = [drop_pct]
    extrapol_pct = drop_pct
    
    # Run the experiment
    %run "{baseline_file}.ipynb"
    
    # Extract average success from subgraph analysis
    subgraph_file = f"sub_{baseline_file}.log"
    if os.path.exists(subgraph_file):
        with open(subgraph_file, 'r') as f:
            content = f.read()
        avg_match = re.search(r'Average attacker success across subgraphs: ([0-9.]+)', content)
        if avg_match:
            avg_success = float(avg_match.group(1))
            results[drop_pct] = avg_success
            diff = avg_success - baseline_success if baseline_success else "N/A"
            print(f"Drop %: {drop_pct*100}%, Avg Success: {avg_success}, Diff: {diff}")
    else:
        print(f"Warning: Could not find subgraph log file")


RUNNING EXPERIMENT WITH DROP PERCENTAGE: 5.0%

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports
create subsgraph for defender concluded
We start with the baseline graph calculation!
Now we are going to run the subgraph analysis
[1] "experiment_3.log"
[1] "Sat Mar 22 17:50:47 2025"



BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE



++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
1 0.000000e+00
10 0.000000e+00
11 2.720264e-01
15 1.852234e-01
2 0.000000e+00
3 0.000000e+00
4 0.000000e+00
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.427503e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.1441309
3 0.3573620
4 0.0000000
5 0.0000000
6 0.0000000
7 0.0000000
8 0.0000000
9 0.0000000
10

In [324]:
# Create DataFrame
df = pd.DataFrame({
    'Drop Percentage': [k*100 for k in results.keys()],
    'Average Success': list(results.values()),
    'Baseline': [baseline_success] * len(results),
    'Difference': [results[k] - baseline_success for k in results.keys()]
})

print("\nFinal Results:")
print(df)
df.to_csv('drop_percentage_results.csv', index=False)


Final Results:
   Drop Percentage  Average Success  Baseline  Difference
0              5.0            0.102     0.089       0.013
1             10.0            0.105     0.089       0.016
2             15.0            0.105     0.089       0.016
3             20.0            0.125     0.089       0.036
4             25.0            0.135     0.089       0.046
5             30.0            0.133     0.089       0.044
6             35.0            0.152     0.089       0.063


In [325]:
# results_df = pd.DataFrame(list(results.items()), columns=['Drop Percentage', 'Attacker Success'])
# print("\nFinal Results:")
# print(results_df)

# # Save results to file
# results_df.to_csv('drop_percentage_results.csv', index=False)

# # Plot results
# plt.figure(figsize=(10, 6))
# plt.plot(results_df['Drop Percentage'] * 100, results_df['Attacker Success'], 'o-', linewidth=2)
# plt.xlabel('Percentage of Nodes Dropped (%)')
# plt.ylabel('Attacker Success Rate')
# plt.title('Impact of Network Visibility Loss on Attacker Success')
# plt.grid(True)
# plt.savefig('drop_percentage_analysis.png')
# plt.show()